In [4]:
pip install selenium

In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects, RequestException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import concurrent.futures

In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def scrape_page(url):
    data = {}
    try:

        # response = requests.get(url)
        # if response.status_code != 200:
        #     print(f"Failed to retrieve the page. Status code: {response.status_code}")
        #     return None

        # Proceed with Selenium if the status is OK
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-extensions')
        options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
        # Add any required options here
        browser = webdriver.Chrome(options=options)

        browser.get(url)

        # Wait for the page to load
        wait = WebDriverWait(browser, 10)
        element_present = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'cssen-1wavjlf')))
        # Continue with your scraping logic

        # Scroll down incrementally
        # for i in range(10):  # Scroll down 10 times
        #     browser.execute_script("window.scrollBy(0, 800);")
            # time.sleep(0.5)  # Adjust sleep time if needed

        # Get the rendered HTML
        html = browser.page_source

        # Initiate data object
        data = {'source': url}

        # Parse the HTML content
        soup = BeautifulSoup(html, 'html.parser')

        # header that contains the price and location string
        body = soup.find('div', attrs={'class': 'cssen-1wavjlf'})

        # properties data
        general_data = body.find('div', attrs={'class': 'cssen-isbt42'})
        properties = general_data.find_all('div', attrs={'class': 'cssen-1bhwfz0'})
        for prop in properties:
            ps = prop.find_all('p')
            key = ps[0].text.strip().replace(':', '')
            value = ps[1].text.strip()
            data[key] = value

        # price
        price = body.find('p', attrs={'class': 'cssen-j3qa3r'})
        data['price'] = price.text.strip()

        # location
        location = body.find('div', attrs={'class': 'cssen-6lzzp6'})
        location_str = location.find_all('p')
        location_str = f'{location_str[0].text.strip()}, {location_str[1].text.strip()}'
        data['location'] = location_str

        amenities = body.find_all('div', attrs={'class': 'cssen-14zw9uq'})

        amenities_concat = ''

        for amenity in amenities:
            amenities_concat += amenity.text.strip(u'\u200b') + ';'

        data['amenities'] = amenities_concat

        description = body.find('p', attrs={'class': 'cssen-1ber2fy'})
        data['description'] = description.text.strip()

        reference = body.find('div', attrs={'class': 'cssen-164krv5'})
        reference = reference.find_all('p')
        data[reference[0].text.strip()] = reference[1].text.strip()

        return data
    finally:
        browser.quit()  # Close the browser even if an exception occurs

    # except TimeoutException:
    #     print("Timed out waiting for page to load")
    # except ConnectionError as e:
    #     print(f"Connection error occurred: {e}")
    # except Timeout as e:
    #     print(f"Timeout error occurred: {e}")
    # except TooManyRedirects as e:
    #     print(f"Too many redirects: {e}")
    # except RequestException as e:
    #     print(f"An error occurred RequestException: {e}")
    # except Exception as e:
    #     print(f"An error occurred: {e}")

    # return None

In [8]:
def scraping_data(path, file_name):
    # Load the CSV file into a DataFrame
    urls = pd.read_csv(path + file_name)
    print(len(urls))
    all_data = []
    failed_url = []
    index = 2
    def process_url(url):
        print(f"Processing {url}")
        return scrape_page(url)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map the process_url function to the list of URLs
        future_to_url = {executor.submit(process_url, url['URL']): url for index, url in urls.iterrows()}

        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                page_data = future.result()
                all_data.append(page_data)
                index += 1
                print(index)
            except Exception as exc:
                print(f"{url['URL']} generated an exception: {exc}")
                failed_url.append(url['URL'])

    # Filter out None values
    filtered_data = [item for item in all_data if item is not None]

    # Create DataFrame
    df = pd.DataFrame(filtered_data)

    csv_file_path = f'{path}data_{file_name}'
    df.to_csv(csv_file_path, index=False)
    return all_data, failed_url

In [9]:
# Replace the path with your file's path
# file_path_rent = '/content/drive/MyDrive/M2 final project/realestate website/'
file_path_sale = '/content/drive/MyDrive/university M2/M2 final project/realestate website/'
d,f = scraping_data(file_path_sale, 'urls_to_be_scraped_rent_3.csv')
# scraping_data(file_path_rent, 'urls_to_be_scraped_rent.csv')

Streaming output truncated to the last 5000 lines.
Processing https://www.realestate.com.lb/en/rent-apartment-house-lebanon/beirut-district/apartment/RWI-RWIS10241378

Processing https://www.realestate.com.lb/en/rent-apartment-house-lebanon/el-metn-district/apartment/MGP-OF-WFD-103
1379
Processing https://www.realestate.com.lb/en/rent-apartment-house-lebanon/beirut-district/apartment/AP-89241380

Processing https://www.realestate.com.lb/en/rent-apartment-house-lebanon/beirut-district/apartment/BL-BL21213741381

Processing https://www.realestate.com.lb/en/rent-apartment-house-lebanon/el-metn-district/apartment/CP-CPCF291382

Processing https://www.realestate.com.lb/en/rent-apartment-house-lebanon/beirut-district/apartment/ZB-ZB-77
1383
Processing https://www.realestate.com.lb/en/rent-apartment-house-lebanon/el-metn-district/apartment/ML-13681
1384
Processing https://www.realestate.com.lb/en/rent-apartment-house-lebanon/beirut-district/apartment/RWI-LS56
1385
Processing https://www.reale